In [1]:
import os
#os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'

os.environ['TF_USE_LEGACY_KERAS'] = '1';

In [2]:
input_default_json_conf_file='fcnn_emotion4_testing_delay_default.json';

In [3]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
import json
from tensorflow.keras.preprocessing.image import load_img

2024-07-28 02:47:18.261530: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-28 02:47:19.427274: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
import sys
sys.path.append('../library');

In [5]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


2024-07-28 02:47:20.647300: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-28 02:47:20.719083: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-28 02:47:20.719320: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [6]:
## Load json conf file
fd = open(os.path.join('./',input_default_json_conf_file));
DATA = json.load(fd);
fd.close()

# Variable globales

In [7]:
## Seed for the random variables
seed_number=0;

## Dataset 
dataset_base_dir     = DATA['dataset_base_dir'];
dataset_labels_file  = DATA['dataset_labels_file'];
dataset_name         = DATA['dataset_name'];

target_labels=['negative','neutro','pain','positive'];

## Model filepath
best_model_file = DATA['model_file'];

## times to count time
times = DATA['times'];

NCOD=DATA['ncod'];

## Output
output_base_dir = DATA['output_base_dir'];

##############################################

# If command line

In [8]:
for n in range(len(sys.argv)):
    if   sys.argv[n]=='--dataset-dir':
        dataset_base_dir=sys.argv[n+1];
    elif sys.argv[n]=='--dataset-file':
        dataset_labels_file=sys.argv[n+1];
    elif sys.argv[n]=='--dataset-name':
        dataset_name=sys.argv[n+1];
    elif sys.argv[n]=='--ncod':
        NCOD=int(sys.argv[n+1]);
    elif sys.argv[n]=='--model-file':
        best_model_file=sys.argv[n+1];
    elif sys.argv[n]=='--times':
        times=int(sys.argv[n+1]);
    elif sys.argv[n]=='--output-dir':
        output_base_dir=sys.argv[n+1];


print('   dataset_base_dir:',dataset_base_dir);
print('dataset_labels_file:',dataset_labels_file);
print('       dataset_name:',dataset_name);
print('               NCOD:',NCOD);
print('    best_model_file:',best_model_file);
print('              times:',times);
print('    output_base_dir:',output_base_dir);



   dataset_base_dir: /media/fernando/Expansion/DATASET/TESE/BER/BER2024/BER2024-SKELETON
dataset_labels_file: test.csv
       dataset_name: ber2024-skel
               NCOD: 20
    best_model_file: /media/fernando/Expansion/OUTPUTS/DOCTORADO2/fcnn_emotion4/ber2024-skel/training_validation_holdout/onlycls_ncod20/model_onlycls_ncod20.h5
              times: 10
    output_base_dir: /media/fernando/Expansion/OUTPUTS/DOCTORADO2/fcnn_emotion4_1


# Set seed of random variables


In [9]:
np.random.seed(seed_number)
tf.keras.utils.set_random_seed(seed_number);

# Loading data of dataset

In [10]:
# Load filenames and labels
test_df = pd.read_csv(os.path.join(dataset_base_dir,dataset_labels_file));
print(test_df)

              d0         d1        d2         d3         d4        d5  \
0      578.53033  230.53235  0.997254  584.17610  221.87859  0.972028   
1      920.09784  375.53006  0.972105  944.41710  346.81830  0.997568   
2      766.30960  252.15428  0.994588  777.61940  225.52296  0.991696   
3      710.24036  232.16788  0.993728  724.85980  201.97440  0.998824   
4      743.05970  192.98499  0.992488  765.76780  169.78462  0.989598   
...          ...        ...       ...        ...        ...       ...   
16674  439.34560  229.46915  0.991680  457.49250  200.63686  0.990643   
16675  831.20184  257.23987  0.993503  870.40430  225.87146  0.991977   
16676  675.22986  146.00938  0.998339  714.07970  129.44043  0.990368   
16677  365.72820  188.31245  0.993100  383.71097  165.13180  0.991528   
16678  820.39874  276.85623  0.992258  846.90924  255.66278  0.990174   

              d6          d7        d8          d9  ...  d42  d43  d44  d45  \
0      555.77313  209.135130  0.985637     0

# Creating output directory

In [11]:

output_dir = os.path.join(output_base_dir,dataset_name,'test_delay','onlycls_ncod'+str(NCOD));

os.makedirs(output_dir,exist_ok = True);

# Create new model

In [12]:
import SkeletonEmotion4Lib.lib_model as mpp

model = mpp.create_model_onlycls(load_weights=False,file_of_weight=best_model_file,ncod=NCOD);
model.summary()

mpp.save_model_parameters(model, os.path.join(output_dir,'parameters_stats.m'));


Loading the weights in: /media/fernando/Expansion/OUTPUTS/DOCTORADO2/fcnn_emotion4/ber2024-skel/training_validation_holdout/onlycls_ncod20/model_onlycls_ncod20.h5
Loaded the weights in: /media/fernando/Expansion/OUTPUTS/DOCTORADO2/fcnn_emotion4/ber2024-skel/training_validation_holdout/onlycls_ncod20/model_onlycls_ncod20.h5
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 4)                 12218     
                                                                 
Total params: 12218 (47.73 KB)
Trainable params: 12218 (47.73 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


2024-07-28 02:47:21.761491: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-28 02:47:21.761722: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-28 02:47:21.761895: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

# Creating numpy vector

In [13]:
import pandas as pd
import SkeletonEmotion4Lib.lib_tools as mpt

np_vector=[];
labelid_vector=[]
for index, row in test_df.iterrows():
    vec=row.iloc[:51].values;
    if vec.dtype != np.float32:
        vec = vec.astype(np.float32);
    np_vector.append(mpt.vector_normalize_coordinates(vec));
    labelid_vector.append(target_labels.index(row.iloc[51].lower()));

print(np_vector[0])
print(labelid_vector[0])

# Time

In [14]:
from pytictoc import TicToc
t = TicToc() 

# Evaluate best model

In [15]:

from tqdm import tqdm

Count=0;
L=len(np_vector);
print('L:',L)


t.tic();
for m in range(times):
    print('m:',m)
    for n in tqdm(range(L)):
        res=mpp.evaluate_model_from_npvector(model, np_vector[n])
        if labelid_vector[n]==res:
            Count=Count+1;
t0=t.tocvalue();


results = dict();
results['categorical_accuracy']=Count*1.0/(L*times);
results['delayms']=t0*1000.0/(times*L);
results['number_of_parameters']=mpp.get_model_parameters(model);
results['ncod']=NCOD;

print(json.dumps(results, indent=4))


m: 0


KeyboardInterrupt: 

In [ ]:
# final all json
with open(os.path.join(output_dir,"testing_data_results.json"), 'w') as f:
    json.dump(results, f,indent=4);

tf.keras.backend.clear_session();
